#### Feature Engineering  

Dataset: 
- _aisles_clean.csv_
- _departments_clean.csv_
- _orders_clean.csv_
- _order_products_clean.csv_
- _products_clean_

Author: Luis Sergio Pastrana Lemus  
Date: 2025-05-05

## __1. Libraries__

In [1]:
from IPython.display import display, HTML
import os
import pandas as pd
from pathlib import Path
import sys

# Define project root dynamically, gets the current directory from whick the notebook belongs and moves one level upper
project_root = Path.cwd().parent

# Add src to sys.path if it is not already
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import function directly (more controlled than import *)
from src import *

## __2. Path to Data file__

In [2]:
# Build route to data file and upload
data_file_path = project_root / "data" / "processed"
df_aisles = load_dataset_from_csv(data_file_path, "aisles_clean.csv", sep=',', header='infer')
df_departments = load_dataset_from_csv(data_file_path, "departments_clean.csv", sep=',', header='infer')
df_order_products = load_dataset_from_csv(data_file_path, "order_products_clean.csv", sep=',', header='infer')
df_orders = load_dataset_from_csv(data_file_path, "orders_clean.csv", sep=',', header='infer')
df_products = load_dataset_from_csv(data_file_path, "products_clean.csv", sep=',', header='infer')


In [3]:
# Format notebook output
format_notebook()

## 3 __Casting to data types__

### 3.1 Casting to string data type

In [4]:
# df_aisles 'aisles' to string
df_aisles['aisle'] = df_aisles['aisle'].astype('string')
df_aisles['aisle'].dtypes

string[python]

In [5]:
# df_departments 'department' to string
df_departments['department'] = df_departments['department'].astype('string')
df_departments['department'].dtypes

string[python]

In [6]:
# df_products 'product_name' to string
df_products['product_name'] = df_products['product_name'].astype('string')
df_products['product_name'].dtypes

string[python]

### 3.2 Casting to numeric data type

In [7]:
# df_order_products 'add_to_cart_order' to int
find_fail_conversion_to_numeric(df_order_products, 'add_to_cart_order')

In [8]:
df_orders_products = convert_object_to_numeric(df_order_products, type='float', include=['add_to_cart_order'])
df_order_products['add_to_cart_order'] = df_order_products['add_to_cart_order'].astype('Int64')
df_order_products['add_to_cart_order'].dtypes

Int64Dtype()

### 3.3 Casting to category data type

In [9]:
# df_orders 'order_dow' to category
df_orders = convert_numday_strday(df_orders, ['order_dow'])
df_orders['order_dow'] = df_orders['order_dow'].astype('category')
df_orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1515936,183418,11,saturday,13,30
1,1690866,163593,5,friday,12,9
2,1454967,39980,4,friday,19,2
3,1768857,82516,56,sunday,20,10
4,3007858,196724,2,thursday,12,17
...,...,...,...,...,...,...
478947,3210681,5617,5,monday,14,7
478948,3270802,112087,2,wednesday,13,6
478949,885349,82944,16,tuesday,11,6
478950,216274,4391,3,wednesday,8,8


In [10]:
df_orders['order_dow'].dtypes

CategoricalDtype(categories=['friday', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday'], ordered=False, categories_dtype=object)

In [11]:
# df_orders 'order_hour_of_day' to category
df_orders['order_hour_of_day'] = df_orders['order_hour_of_day'].astype('category')
df_orders['order_hour_of_day'].dtypes


CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], ordered=False, categories_dtype=int64)

In [12]:
# df_products 'aisle_id' to category
df_products['aisle_id'] = df_products['aisle_id'].astype('category')
df_products['aisle_id'].dtypes

CategoricalDtype(categories=[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
                  ...
                  125, 126, 127, 128, 129, 130, 131, 132, 133, 134],
, ordered=False, categories_dtype=int64)

In [13]:
# df_products 'department_id' to category
df_products['department_id'] = df_products['department_id'].astype('category')
df_products['department_id'].dtypes

CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], ordered=False, categories_dtype=int64)

## 4. Feature Engineering

### 4.1 Order activity variation

#### 4.1.1 Order activity by day

In [14]:
df_orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1515936,183418,11,saturday,13,30
1,1690866,163593,5,friday,12,9
2,1454967,39980,4,friday,19,2
3,1768857,82516,56,sunday,20,10
4,3007858,196724,2,thursday,12,17
...,...,...,...,...,...,...
478947,3210681,5617,5,monday,14,7
478948,3270802,112087,2,wednesday,13,6
478949,885349,82944,16,tuesday,11,6
478950,216274,4391,3,wednesday,8,8


In [17]:
# Days with most order activity
pivot_orders_activity_day = pd.pivot_table(df_orders, index='order_dow', values='order_id', aggfunc='count', observed=True)
pivot_orders_activity_day = pivot_orders_activity_day.reset_index()
pivot_orders_activity_day


,order_dow,order_id
0,friday,63488
1,monday,82185
2,saturday,62649
3,sunday,84090
4,thursday,59810
5,tuesday,65833
6,wednesday,60897


In [20]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "order_activity" / "orders_activity_day.csv"

pivot_orders_activity_day.to_csv(processed_path, index=False)

#### 4.1.2 Order activity by time

In [19]:
# Hours with most order activity
pivot_orders_activity_time = pd.pivot_table(df_orders, index='order_hour_of_day', values='order_id', aggfunc='count', observed=True)
pivot_orders_activity_time = pivot_orders_activity_time.reset_index()
pivot_orders_activity_time

,order_hour_of_day,order_id
0,0,3180
1,1,1763
2,2,989
3,3,770
4,4,765
5,5,1371
6,6,4215
7,7,13043
8,8,25024
9,9,35896


In [21]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "order_activity" / "orders_activity_time.csv"

pivot_orders_activity_time.to_csv(processed_path, index=False)

#### 4.1.3 Order activity frequency

In [22]:
# Tipical order frequency
df_orders_filtered = df_orders.loc[(df_orders['days_since_prior_order'] > 0)]
pivot_orders_activity_frequency = pd.pivot_table(df_orders, index=['user_id'], values='days_since_prior_order', aggfunc='mean')
pivot_orders_activity_frequency = pivot_orders_activity_frequency.rename(columns={'days_since_prior_order': 'avg_frequency_order'})
pivot_orders_activity_frequency

,avg_frequency_order
user_id,
2,4.000000
4,24.500000
5,19.000000
6,11.000000
7,4.000000
...,...
206203,30.000000
206206,5.000000
206207,9.400000


In [23]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "order_activity" / "orders_activity_frequency.csv"

pivot_orders_activity_frequency.to_csv(processed_path, index=False)

### 4.2 Product variation

In [24]:
df_order_products

,order_id,product_id,add_to_cart_order,reordered
0,2141543,11440,17,False
1,567889,1560,1,True
2,2261212,26683,1,True
3,491251,8670,35,True
4,2571142,1940,5,True
...,...,...,...,...
4545002,577211,15290,12,True
4545003,1219554,21914,9,False
4545004,692640,47766,4,True
4545005,319435,691,8,True


#### 4.2.1 Product purchasing quantity

In [27]:
# Number of products within an order
pivot_product_purchase_quantity = pd.pivot_table(df_order_products, index='order_id', values='product_id', aggfunc='count')
pivot_product_purchase_quantity = pivot_product_purchase_quantity.rename(columns={'product_id': 'total_products'})
pivot_product_purchase_quantity = pivot_product_purchase_quantity.reset_index()
pivot_product_purchase_quantity

,order_id,total_products
0,4,13
1,9,15
2,11,5
3,19,3
4,20,8
...,...,...
450041,3421034,17
450042,3421053,9
450043,3421071,5
450044,3421077,4


In [28]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "product_activity" / "product_purchase_quantity.csv"

pivot_product_purchase_quantity.to_csv(processed_path, index=False)

#### 4.2.2 Product repurchase rate

In [29]:
# Products with highest repurchase rate
pivot_product_repurchase = pd.pivot_table(df_order_products, index='product_id', values='reordered', aggfunc=['sum', 'count'])
pivot_product_repurchase.columns = ['times_reordered', 'total_orders']
pivot_product_repurchase

,times_reordered,total_orders
product_id,,
1,158,280
2,0,11
3,31,42
4,25,49
7,1,2
...,...,...
49690,4,5
49691,31,72
49692,5,12


In [30]:
pivot_product_repurchase['repurchase_rate'] = (pivot_product_repurchase['times_reordered'] / pivot_product_repurchase['total_orders'])
pivot_product_repurchase

,times_reordered,total_orders,repurchase_rate
product_id,,,
1,158,280,0.564286
2,0,11,0.000000
3,31,42,0.738095
4,25,49,0.510204
7,1,2,0.500000
...,...,...,...
49690,4,5,0.800000
49691,31,72,0.430556
49692,5,12,0.416667


In [31]:
pivot_product_repurchase = pivot_product_repurchase.reset_index()
pivot_product_repurchase

,product_id,times_reordered,total_orders,repurchase_rate
0,1,158,280,0.564286
1,2,0,11,0.000000
2,3,31,42,0.738095
3,4,25,49,0.510204
4,7,1,2,0.500000
...,...,...,...,...
45568,49690,4,5,0.800000
45569,49691,31,72,0.430556
45570,49692,5,12,0.416667
45571,49693,11,25,0.440000


In [32]:
pivot_product_repurchase = pivot_product_repurchase.merge(df_products[['product_id', 'product_name']], on='product_id', how='left')
pivot_product_repurchase

,product_id,times_reordered,total_orders,repurchase_rate,product_name
0,1,158,280,0.564286,chocolate_sandwich_cookies
1,2,0,11,0.000000,all_seasons_salt
2,3,31,42,0.738095,robust_golden_unsweetened_oolong_tea
3,4,25,49,0.510204,smart_ones_classic_favorites_mini_rigatoni_wit...
4,7,1,2,0.500000,pure_coconut_water_with_orange
...,...,...,...,...,...
45568,49690,4,5,0.800000,high_performance_energy_drink
45569,49691,31,72,0.430556,original_pancake_waffle_mix
45570,49692,5,12,0.416667,organic_instant_oatmeal_light_maple_brown_sugar
45571,49693,11,25,0.440000,spring_water_body_wash


In [33]:
pivot_product_repurchase.sort_values(by=['repurchase_rate', 'times_reordered'], ascending=[False, False]).head(20)

,product_id,times_reordered,total_orders,repurchase_rate,product_name
41269,45035,17,17,1.0,coffee_flavor_yogurt
35935,39210,16,16,1.0,super_premium_chocolate_ice_cream
40400,44086,16,16,1.0,natural_fat_free_cheddar_shredded_cheese
40224,43895,14,14,1.0,sugar_land_sweet_tea
1523,1673,13,13,1.0,free_ride_hibiscus_brew_craft_brewed_kombucha
42174,46015,12,12,1.0,100_colombia_supremo_medium
9538,10463,11,11,1.0,sonic_energy_refreshed
9863,10813,11,11,1.0,jumbo_ranch_sunflower_seeds
34875,38041,11,11,1.0,orange_sparkling_water_beverage
5964,6530,10,10,1.0,organic_popped_corn_simply_salted


In [34]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "product_activity" / "product_repurchase.csv"

pivot_product_repurchase.to_csv(processed_path, index=False)

#### 4.2.3 Product preference and relevance

In [35]:
# Products typically added to the cart first
product_preference = (df_order_products[df_order_products['add_to_cart_order'] == 1].groupby('product_id').size()
                      .reset_index(name='first_add_count').sort_values(by='first_add_count', ascending=False))
product_preference

,product_id,first_add_count
13414,24852,15562
7127,13176,11026
15028,27845,4363
11384,21137,3946
25439,47209,3390
...,...,...
26,43,1
25,41,1
20,33,1
11804,21942,1


In [36]:
product_preference = product_preference.merge(df_products[['product_id', 'product_name']], on='product_id', how='left')
product_preference


,product_id,first_add_count,product_name
0,24852,15562,banana
1,13176,11026,bag_of_organic_bananas
2,27845,4363,organic_whole_milk
3,21137,3946,organic_strawberries
4,47209,3390,organic_hass_avocado
...,...,...,...
26755,43,1,organic_clementines
26756,41,1,organic_sourdough_einkorn_crackers_rosemary
26757,33,1,organic_spaghetti_style_pasta
26758,21942,1,dry_roasted_edamame_berry_blend


In [37]:
# Products typically added to the cart first relevance
total_orders = df_order_products.groupby('product_id').size().reset_index(name='total_orders')
product_preference = product_preference.merge(total_orders, on='product_id')
product_preference['first_add_rate'] = (product_preference['first_add_count'] / product_preference['total_orders'])
product_preference


,product_id,first_add_count,product_name,total_orders,first_add_rate
0,24852,15562,banana,66050,0.235609
1,13176,11026,bag_of_organic_bananas,53297,0.206878
2,27845,4363,organic_whole_milk,19600,0.222602
3,21137,3946,organic_strawberries,37039,0.106536
4,47209,3390,organic_hass_avocado,29773,0.113862
...,...,...,...,...,...
26755,43,1,organic_clementines,29,0.034483
26756,41,1,organic_sourdough_einkorn_crackers_rosemary,6,0.166667
26757,33,1,organic_spaghetti_style_pasta,5,0.200000
26758,21942,1,dry_roasted_edamame_berry_blend,1,1.000000


In [38]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "product_activity" / "product_preference.csv"

product_preference.to_csv(processed_path, index=False)

#### 4.2.4 Product relationship between the order in which a product is added to the cart and the likelihood of it being reordered

In [39]:
# Relationship between the order in which a product is added to the cart and the likelihood of it being reordered
df_cart_order_reorder = df_order_products[['add_to_cart_order', 'reordered']].copy()
df_cart_order_reorder

,add_to_cart_order,reordered
0,17,False
1,1,True
2,1,True
3,35,True
4,5,True
...,...,...
4545002,12,True
4545003,9,False
4545004,4,True
4545005,8,True


In [40]:
reorder_rate_by_position = (df_cart_order_reorder.groupby('add_to_cart_order')['reordered'].agg(['mean', 'count']).reset_index()
                            .rename(columns={'mean': 'reorder_rate', 'count': 'product_count'}))
reorder_rate_by_position

,add_to_cart_order,reorder_rate,product_count
0,1,0.678620,450046
1,2,0.677059,428199
2,3,0.658986,401907
3,4,0.637511,372861
4,5,0.617483,341807
...,...,...,...
59,60,0.472441,127
60,61,0.359649,114
61,62,0.448980,98
62,63,0.471264,87


In [41]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "product_activity" / "product_reorder_rate_by_position.csv"

reorder_rate_by_position.to_csv(processed_path, index=False)

#### 4.2.5 Product repurchase frequency among different user segments

In [42]:
# Repurchase frequency among different user segments
df_op_user = df_order_products.merge(df_orders[['order_id', 'user_id']], on='order_id', how='left')
df_op_user

,order_id,product_id,add_to_cart_order,reordered,user_id
0,2141543,11440,17,False,58454
1,567889,1560,1,True,129474
2,2261212,26683,1,True,47476
3,491251,8670,35,True,134505
4,2571142,1940,5,True,155995
...,...,...,...,...,...
4545002,577211,15290,12,True,76414
4545003,1219554,21914,9,False,163902
4545004,692640,47766,4,True,597
4545005,319435,691,8,True,7124


In [43]:
user_repurchase_rate = (df_op_user.groupby('user_id')['reordered'].agg(['sum', 'count'])  # sum = total reorders, count = total products
                       .rename(columns={'sum': 'total_reorders', 'count': 'total_products'}))

user_repurchase_rate['repurchase_rate'] = (user_repurchase_rate['total_reorders'] / user_repurchase_rate['total_products'])
user_repurchase_rate

,total_reorders,total_products,repurchase_rate
user_id,,,
2,1,26,0.038462
4,0,2,0.000000
5,8,12,0.666667
6,0,4,0.000000
7,13,14,0.928571
...,...,...,...
206203,6,27,0.222222
206206,15,21,0.714286
206207,41,46,0.891304


In [44]:
def label_segment(rate):
    if rate <= 0.25:
        return 'Low'
    elif rate <= 0.5:
        return 'Moderate'
    elif rate <= 0.75:
        return 'High'
    else:
        return 'Very High'

user_repurchase_rate['repurchase_segment'] = user_repurchase_rate['repurchase_rate'].apply(label_segment)
user_repurchase_rate

,total_reorders,total_products,repurchase_rate,repurchase_segment
user_id,,,,
2,1,26,0.038462,Low
4,0,2,0.000000,Low
5,8,12,0.666667,High
6,0,4,0.000000,Low
7,13,14,0.928571,Very High
...,...,...,...,...
206203,6,27,0.222222,Low
206206,15,21,0.714286,High
206207,41,46,0.891304,Very High


In [45]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "product_activity" / "product_user_segment_repurchase_rate.csv"

user_repurchase_rate.to_csv(processed_path, index=False)